In [ ]:
import base64
import requests
import json
import os

## URL de autenticação/token
token_url = 'https://www.bling.com.br/Api/v3/oauth/token'

## Credenciais da aplicação
client_id = 'CLIENT_ID'
client_secret = 'CLIENT_SECRET'
credentials = f'{client_id}:{client_secret}'
base64_credentials = base64.b64encode(credentials.encode('ascii')).decode()

## Arquivo onde os tokens serão armazenados
tokens_file = 'tokens.json'

def save_tokens(access_token, refresh_token):
    """Salva os tokens em um arquivo JSON."""
    tokens = {
        'access_token': access_token,
        'refresh_token': refresh_token
    }
    with open(tokens_file, 'w') as f:
        json.dump(tokens, f)

def get_tokens():
    """Obtém os tokens do arquivo JSON, se existir."""
    if os.path.exists(tokens_file):
        with open(tokens_file, 'r') as f:
            tokens = json.load(f)
            return tokens['access_token'], tokens['refresh_token']
    else:
        return None, None

## Obtendo os tokens do armazenamento
access_token, refresh_token = get_tokens()

if not access_token or not refresh_token:
    ## Caso os tokens não estejam armazenados ou estejam vazios, é necessário autorizar o aplicativo
    print('Autorize o aplicativo e obtenha os tokens.')

else:
    ## Caso os tokens estejam armazenados, faz a requisição GET à API usando o token de acesso
    api_url = 'https://www.bling.com.br/Api/v3/empresas/me'
    headers = {'Authorization': f'Bearer {access_token}'}

    try:
        response = requests.get(api_url, headers=headers)

        if response.status_code == 200:
            ## Requisição bem-sucedida, faça aqui o processamento dos dados retornados pela API
            print('Requisição bem-sucedida. Processando dados...')

        elif response.status_code == 401:
            ## O token expirou, obtém um novo token de acesso usando o Refresh Token
            print('Obtendo novo token de acesso...')
            data = {'grant_type': 'refresh_token', 'refresh_token': refresh_token}
            headers = {'Content-Type': 'application/x-www-form-urlencoded', 'Accept': '1.0', 'Authorization': f'Basic {base64_credentials}'}
            response = requests.post(token_url, data=data, headers=headers)

            if response.status_code == 200:
                ## Requisição bem-sucedida, obtenha os novos tokens
                response_data = response.json()
                access_token = response_data['access_token']
                refresh_token = response_data['refresh_token']

                ## Atualizando os tokens no armazenamento
                save_tokens(access_token, refresh_token)

                ## Faça aqui o processamento dos dados retornados pela API
                print('Novo token de acesso obtido. Requisição bem-sucedida.')

            else:
                ## Algum erro ocorreu na requisição para obter o novo token
                print(f'Erro na requisição para obter novo token - Código: {response.status_code}, Mensagem: {response.text}')

        else:
            ## Algum erro ocorreu na requisição GET à API
            print(f'Erro na requisição GET - Código: {response.status_code}, Mensagem: {response.text}')

    except requests.exceptions.RequestException as e:
        ## Trata exceções de erro de rede, por exemplo, se a API estiver inacessível
        print(f'Erro na requisição: {e}')